In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


# Download new dataset

In [2]:
import os

In [3]:
# Mount Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Change these
kaggle_dataset_path = "yashvi/number-captcha" 
gdrive_path = "/content/gdrive/MyDrive/Captcha_Datasets"
# Example datasets:
# fournierp/captcha-version-2-images
# fanbyprinciple/captcha-images
# yashvi/number-captcha

# Sets Kaggle Config Environment Variable (Path of Kaggle.json file)
os.environ['KAGGLE_CONFIG_DIR'] = gdrive_path
%cd $gdrive_path

# organize datasets into folders (by username)
path = kaggle_dataset_path.lower().split("/")[0]
%mkdir -p $path
if os.path.exists(path):
  input("Directory already exists. Continue downloading?")

# Fetch the CAPTCHA Dataset
!kaggle datasets download -d $kaggle_dataset_path -p $path

# Then Unzip the Dataset
n_files = !unzip -l $path/*.zip | grep .png | wc -l
!unzip $path/*.zip  -d $path | pv -l -s {n_files[0]} > /dev/null


# Process data

In [138]:
import cv2 as cv
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from functools import reduce

In [182]:
# Works specifically for the fournierp dataset
dataset_path = "/content/gdrive/MyDrive/Captcha_Datasets/fournierp/samples/"
processed_data_path = "/content/gdrive/MyDrive/Captcha_Datasets/fournierp/processed/"
%mkdir -p /content/gdrive/MyDrive/Captcha_Datasets/fournierp/processed/{0..9} 
%mkdir -p /content/gdrive/MyDrive/Captcha_Datasets/fournierp/processed/{a..z}

"""
Produces no more than 4 images per character in each captcha.
"""
def process_fournierp_image(files, outputs=False, result=False, wiggle=True):
  img = cv.imread(dataset_path+files, cv.IMREAD_GRAYSCALE)
  label = files.split(".")[0]
  # threshold and invert
  ret,thresh = cv.threshold(img,127,255,cv.THRESH_BINARY_INV)
  # VPP and threshold on VPP
  vpp = np.sum(thresh, axis=0) # vertical projection profile: sum of pixels vertically
  vpp_threshold = 0.2 * 255 * thresh.shape[0] # threshold of 20% of height
  chars = np.where(vpp > vpp_threshold, thresh, np.zeros(img.shape))
  vpp_thresh_diff = np.diff(vpp > vpp_threshold)
  diff_indices = np.asarray(vpp_thresh_diff).nonzero()[0] # where vpp <= threshold
  # truncate image on both sides
  assert diff_indices.size >= 2
  right_trunc = diff_indices[-1] # usually no problem
  left_trunc = diff_indices[0] if diff_indices.size == 2 or \
              diff_indices[2] - diff_indices[0] > (right_trunc - diff_indices[0])/(len(label)*1.3) \
              else diff_indices[2] # basically, we might want to truncate to the end of the second grey chunk 
  trunc = thresh[:,left_trunc:right_trunc]
  # Outputs to show what's happening
  if outputs:
    print("filename:", files)
    cv2_imshow(img)
    cv2_imshow(thresh)
    cv2_imshow(np.where(vpp > vpp_threshold, img, 100*np.ones(img.shape)))
    plt.figure()
    plt.title("VPP")
    plt.plot(vpp)
    plt.figure()
    plt.title("VPP thresholded")
    plt.plot(vpp > vpp_threshold)
    cv2_imshow(trunc)

  for i, char in enumerate(label):
    left = int(i*trunc.shape[1]/len(label))
    right = int((i+1)*trunc.shape[1]/len(label))
    char_img = trunc[:,left:right]
    if result:
      cv2_imshow(char_img)
    save_char_img(char, char_img)
    # first and last characters will produce only 2 wiggles
    if wiggle:
      if i != 0 and i < len(label):
        larger = trunc[:, int(left-0.02*trunc.shape[1]):int(right+0.02*trunc.shape[1])]
        save_char_img(char, larger)
      wiggle1 = trunc[:, int(left-0.05*trunc.shape[1]):int(right-0.05*trunc.shape[1])] if i != 0 else \
                trunc[:, left:int(right-0.05*trunc.shape[1])]
      save_char_img(char, wiggle1)
      wiggle2 = trunc[:, int(left+0.05*trunc.shape[1]):int(right+0.05*trunc.shape[1])] if i != len(label)-1 else \
                trunc[:, int(left+0.05*trunc.shape[1]):right]
      save_char_img(char, wiggle2)
      if result:
        if i != 0 and i < len(label):
          cv2_imshow(larger)
        cv2_imshow(wiggle1)
        cv2_imshow(wiggle2)

"""
Name the new file {char}-{count}.png where count is the highest number after '-' of existing files.
eg. for a folder with 2-0.png 2-1.png 2-23.png, we will save 2-24.png
"""
def save_char_img(char, char_img):
    same_chars = list(filter(lambda a: not a.startswith('.'), os.listdir(processed_data_path+char)))
    nums = map(lambda a: int(a.split("-")[1].split(".")[0]), same_chars)
    char_count = 0 if len(same_chars) == 0 else max(nums) + 1
    cv.imwrite(processed_data_path+char+"/"+char+"-"+str(char_count)+".png", char_img)


  # kernel = np.ones((5,5), np.uint8)
  
  # #Operations on the image to cleanup and remove noise
  # gaussian_blur = cv.GaussianBlur(res,(5,5),0)
  # ret, thresh = cv.threshold(gaussian_blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
  
  # thresh = cv.erode(thresh, kernel, iterations=2)
  # thresh = cv.dilate(thresh, kernel, iterations=2)
  # thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=2)
  # cv.imwrite(processed_data_path + files, res)   
  # cv.imshow(processed_data_path + files)

In [183]:
# Try a sample
process_fournierp_image("226md.png", result=True)

0
1
2


3
4
5
6


0
1
2
3


0
1
2
3


0
1
2
3


In [188]:
# Iterate over all files
for files in os.listdir(dataset_path):
  if files.endswith(".png"):    
    process_fournierp_image(files)

Streaming output truncated to the last 5000 lines.
796
797
798
1561
1562
1563
730
731
732
733
624
625
626
627
729
730
731
732
1564
1565
1566
1567
766
767
768
1568
1569
1570
1571
734
735
736
737
1572
1573
1574
1575
799
800
801
802
1576
1577
1578
628
629
630
631
632
633
634
635
704
705
706
707
807
808
809
810
792
793
794
764
765
766
767
1579
1580
1581
1582
803
804
805
806
636
637
638
639
795
796
797
640
641
642
643
544
545
546
547
644
645
646
647
628
629
630
631
733
734
735
648
649
650
651
770
771
772
773
632
633
634
635
736
737
738
739
798
799
800
735
736
737
738
807
808
809
810
774
775
776
777
801
802
803
804
1583
1584
1585
708
709
710
711
768
769
770
771
811
812
813
814
548
549
550
551
740
741
742
811
812
813
814
1586
1587
1588
1589
805
806
807
808
815
816
817
818
809
810
811
819
820
821
822
815
816
817
818
739
740
741
742
778
779
780
781
812
813
814
793
794
795
796
797
798
799
800
712
713
714
715
652
653
654
655
743
744
745
636
637
638
639
815
816
817
818
738
739
740
741
1590
1591
15

In [187]:
%rm /content/gdrive/MyDrive/Captcha_Datasets/fournierp/processed/d/*.png

In [197]:
# frequency of characters?
for dir in os.listdir("/content/gdrive/MyDrive/Captcha_Datasets/fournierp/processed"):
  print (dir, len(os.listdir("/content/gdrive/MyDrive/Captcha_Datasets/fournierp/processed/" + dir)))

0 0
1 0
2 1013
3 1014
4 1090
5 1067
6 972
7 987
8 1003
9 0
a 0
b 910
c 997
d 982
e 897
f 1037
g 1028
h 0
i 0
j 0
k 0
l 0
m 1041
n 2005
o 0
p 955
q 0
r 0
s 0
t 0
u 0
v 0
w 902
x 999
y 880
z 0
